In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Kendral Correlation
- With reference to subin : https://www.kaggle.com/subinium/tps-apr-highlighting-the-data

## <p style="background-color:Salmon; font-family:newtimeroman; font-size:140%; text-align:center; border-radius: 15px 50px;">Table of Content</p>

* [1. Import Required Libraries](#1)
* [2. Import Data](#1)    
* [3. EDA(Exploratory Data Analysis)](#2)
    * [3.1 Missing Values](#2.1)
    * [3.2 Droping Unwanted Columns](#2.2)
    * [3.3 Correlation between Features](#2.3)
       * [3.3.1 The correlation between the continuos variables](#2.4)
       * [3.3.2 The correlation between this continuos features and the target](#2.4)
    * [3.4 Outliers](#2.4)
    * [3.5 Skewness and Kurtsis](#2.4)
    * [3.6 Data Visualisation](#2.4)
* [4. Feature Engineering 🛠](#3)
* [5. Stacking](#4)
* [6. Model Building and Validation](#4)

## <p style="background-color:Salmon; font-family:newtimeroman; font-size:160%; text-align:center; border-radius: 15px 50px;">Features description</p>

|Feature          |Description                                                                      |
|-----------------|---------------------------------------------------------------------------------|
|Pclass           |A proxy for socio-economic status (SES) where 1st = Upper, 2nd = Middle and 3rd = Lower.|
|Sex              |male and female.|
|Age              |fractional if it less than 1 and age estimation in the form of xx.5.|
|SibSp            |number of siblings / spouses aboard the Synthanic; siblings are brother, sister, stepbrother and stepsister and spouses are husband and wife (mistresses and fiancés were ignored).|
|Parch            |# of parents / children aboard the Synthanic; parents are mother and father; child are daughter, son, stepdaughter and stepson. Some children travelled only with a nanny, therefore Parch is 0 for them.|
|Fare             |the paassenger fare.|
|Cabin            |the cabin number.|
|Emarked          |port of embarkation where C is Cherbourg, Q is Queenstown and S is Southampton.|
|Ticket           |ticket number.|
|Name             |passengers name.|
|Survived         |target variable where 0 is not survived and 1 is survived.|

## <p style="background-color:Yellow; font-family:newtimeroman; font-size:160%; text-align:center; border-radius: 15px 50px;">1. Import Required Libraries</p>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import plotly.io as pio
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

plt.style.use("fivethirtyeight")
sns.set_style("darkgrid")

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, auc, roc_curve, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
import lightgbm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

import torch

## <p style="background-color:Yellow; font-family:newtimeroman; font-size:160%; text-align:center; border-radius: 15px 50px;">2. Import Data</p>

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv")

### First 5 rows
- **Print First 5 rows in the train dataset**

In [ ]:
train.head()

- **Print First 5 rows in the test dataset**

In [ ]:
test.head()

- **Print First 5 rows in the submission dataset**

In [ ]:
sub.head()

### Numbers of rows and columns

In [ ]:
print('Rows and Columns in train dataset:', train.shape)
print('Rows and Columns in test dataset:', test.shape)

In [ ]:
trainoriginal = train.copy()
testoriginal = test.copy()

In [ ]:
display(train.info())
display(test.info())

## <p style="background-color:Yellow; font-family:newtimeroman; font-size:160%; text-align:center; border-radius: 15px 50px;">3. EDA(Exploratory Data Analysis)</p>

## <p style="background-color:magenta; font-family:newtimeroman; font-size:130%; text-align:center; border-radius: 15px 50px;">a) Missing Values</p>

- For different background colors : http://www.htmlcodes.ws/color/html-color-code-generator.cfm?colorName=SkyBlue

In [ ]:
print('Missing values in train dataset:', sum(train.isnull().sum()))
print('Missing values in test dataset:', sum(test.isnull().sum()))

In [ ]:
print('Missing values per columns in train dataset')
for col in train.columns:
    train_col = train[col].isnull().sum()
    print(f'{col}: {train_col}')

In [ ]:
print('Missing values per columns in test dataset')
for col in test.columns:
    test_col = test[col].isnull().sum()
    print(f'{col}: {test_col}')

In [ ]:
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_grey = "#c6ccd8"
primary_black = "#202022"

In [ ]:
missing = (train.isna().sum().sort_values(ascending=False) / len(train) * 100)[:6]
fig, ax = plt.subplots(1,1,figsize=(9, 8))

ax.bar(missing.index, 100, color=primary_grey, width=0.8)

bar = ax.bar(missing.index, missing, color=primary_black, width=0.6)
ax.bar_label(bar, fmt='%.01f %%')
ax.spines.left.set_visible(False)
ax.set_yticks([])
ax.set_title('Missing Values Ratio', fontweight='bold')

plt.show()

## <p style="background-color:magenta; font-family:newtimeroman; font-size:120%; text-align:center; border-radius: 15px 50px;">1. Survived</p>

In [ ]:
train['Survived'].value_counts().to_frame()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(train['Survived'])
plt.show()

In [ ]:
def categorical(cat, train=train):
    ax = plt.subplots(figsize=(8,6))
    sns.countplot(x=cat, data=train)
    plt.xticks(size=16)
    plt.yticks(size=16)
    plt.xlabel(cat+' :Count', fontweight='bold', size=17)
    plt.ylabel('Count', fontweight='bold', size=17)
    plt.show()
    
def hue(cat, train=train):
    ax = plt.subplots(figsize=(8,6))
    sns.countplot(x=cat, hue='Survived', data=train)
    plt.show()

## <p style="background-color:magenta; font-family:newtimeroman; font-size:120%; text-align:center; border-radius: 15px 50px;">2. Sex</p>

In [ ]:
categorical('Sex')

In [ ]:
hue('Sex')

## <p style="background-color:magenta; font-family:newtimeroman; font-size:120%; text-align:center; border-radius: 15px 50px;">3. Embarked</p>

In [ ]:
categorical('Embarked')

In [ ]:
hue('Embarked')

## <p style="background-color:magenta; font-family:newtimeroman; font-size:120%; text-align:center; border-radius: 15px 50px;">4. Pclass</p>

In [ ]:
categorical('Pclass')

In [ ]:
hue('Pclass')

## <p style="background-color:magenta; font-family:newtimeroman; font-size:120%; text-align:center; border-radius: 15px 50px;">5. SibSp</p>

In [ ]:
categorical('SibSp')

In [ ]:
hue('SibSp')

## <p style="background-color:magenta; font-family:newtimeroman; font-size:120%; text-align:center; border-radius: 15px 50px;">6. Parch</p>

In [ ]:
categorical('Parch')

In [ ]:
hue('Parch')

## <p style="background-color:magenta; font-family:newtimeroman; font-size:140%; text-align:center; border-radius: 15px 50px;">7. Age</p>

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=train['Age'],
                           name='train', 
                           histnorm='probability density',
                           xbins=dict(
                               start=0,
                               end=100,
                               size=2
                           ),
                           marker_color='lightsalmon',
                           opacity=0.75
                          )
             ) 
fig.add_trace(go.Histogram(x=test['Age'],
                           name='test', 
                           histnorm='probability density',
                           xbins=dict(
                               start=0,
                               end=100,
                               size=2
                           ),
                           marker_color='lightseagreen',
                           opacity=0.75
                          )
             ) 
fig.update_layout(title='Passengers Age Distribution (train-test)',
                  xaxis_title='Age [years]', 
                  yaxis_title='Probability Density [-]',
                  titlefont={'size': 24},
                  font_family = 'San Serif',
                  width=1000,height=800,
                  template="plotly_dark",
                  showlegend=True,
                  paper_bgcolor="black",
                  font=dict(
                      color ='white',
                      ),
                  legend=dict(
                      orientation="v",
                      y=1, 
                      yanchor="top", 
                      x=1.0, 
                      xanchor="right",)   
 )
fig.show()


## <p style="background-color:magenta; font-family:newtimeroman; font-size:130%; text-align:center; border-radius: 15px 50px;">b) Droping Unwanted Columns</p>

In [ ]:
train.drop(['PassengerId'], axis=1, inplace=True)
test.drop(['PassengerId'], axis=1, inplace=True)

## <p style="background-color:magenta; font-family:newtimeroman; font-size:130%; text-align:center; border-radius: 15px 50px;">c) Correlation between Features</p>

## <p style="background-color:GreenYellow; font-family:newtimeroman; font-size:120%; text-align:left; ">i) The correlation between the continuos variables</p>  

 - a. Pearson Correlation
 - b. Spearman Correlation
 - c. kendall

In [ ]:
# Pearson Correlation
plt.figure(figsize=(10,8))
sns.heatmap(train.corr(method='pearson'), cbar=False, annot=True, fmt='.1', linewidth=0.2);

In [ ]:
# Spearman Correlation
plt.figure(figsize=(10,8))
sns.heatmap(train.corr(method='spearman'), cbar=False, annot=True, fmt='.1', linewidth=0.2);

In [ ]:
# kendall
fig, ax = plt.subplots(1, 3, figsize=(17 , 5))

feature_lst = ['Pclass', 'Age', 'SibSp','Parch','Fare']

corr = train[feature_lst].corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


for idx, method in enumerate(['pearson', 'kendall', 'spearman']):
    sns.heatmap(train[feature_lst].corr(method=method), ax=ax[idx],
            square=True, annot=True, fmt='.2f', center=0, linewidth=2,
            cbar=False, cmap=sns.diverging_palette(240, 10, as_cmap=True),
            mask=mask
           ) 
    ax[idx].set_title(f'{method.capitalize()} Correlation', loc='left', fontweight='bold')     

plt.show()

## <p style="background-color:GreenYellow; font-family:newtimeroman; font-size:120%; text-align:left; ">ii) The correlation between this continuos features and the target</p>  

In [ ]:
train.corr()['Survived']

In [ ]:
a = train.drop('Survived', axis=1)
a.corrwith(train['Survived']).plot(kind='bar', grid=True, figsize=(10,8), color='salmon')
plt.title("Correlation with target", fontweight='bold', size=20)
plt.xticks(size=15, rotation=90)
plt.yticks(size=15)
plt.show()

## <p style="background-color:magenta; font-family:newtimeroman; font-size:130%; text-align:center; border-radius: 15px 50px;">d) Outliers</p>

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(data=train, orient='h', palette='Set2');

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(data=test, orient='h', palette='Set2');

## <p style="background-color:magenta; font-family:newtimeroman; font-size:130%; text-align:center; border-radius: 15px 50px;">e) Skewness and Kurtsis</p>

In [ ]:
train.skew().sort_values(ascending=True)

In [ ]:
test.skew().sort_values(ascending=True)

## <p style="background-color:magenta; font-family:newtimeroman; font-size:130%; text-align:center; border-radius: 15px 50px;">f) Data Visualisation</p>

## <p style="background-color:Blue; font-family:newtimeroman; font-size:120%; text-align:left; border-radius: 5px 5px;"> iii) Multivariate Analysis</p>

In [ ]:
sns.pairplot(data=train);

In [ ]:
sns.pairplot(data=test);

## <p style="background-color:Yellow; font-family:newtimeroman; font-size:160%; text-align:center; border-radius: 15px 50px;">4. Feature Engineering</p>

In [ ]:
train = train.drop(['Name', 'Cabin'], axis=1)
test = test.drop(['Name', 'Cabin'], axis=1)

In [ ]:
display(train.head(2))
display(test.head(2))

In [ ]:
# Fill Missing values in train and test
train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(test['Age'].mean())

train['Fare'] = train['Fare'].fillna(train['Fare'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [ ]:
le = LabelEncoder()

train['Sex'] = le.fit_transform(train['Sex'])
train['Embarked'] = le.fit_transform(train['Embarked'])

test['Sex'] = le.fit_transform(test['Sex'])
test['Embarked'] = le.fit_transform(test['Embarked'])

In [ ]:
tickets = train['Ticket'].str.split()
tickets = [
    ticket[-1] if len(ticket) > 1 else ticket[0] 
    for ticket in [
        ['9999999999'] if ticket is np.nan else ticket 
    for ticket in tickets]
]
tickets = [int(elem) if elem.isdigit() else 9999999999 for elem in tickets]
train['Ticket'] = tickets

tickets = test['Ticket'].str.split()
tickets = [
    ticket[-1] if len(ticket) > 1 else ticket[0] 
    for ticket in [
        ['9999999999'] if ticket is np.nan else ticket 
    for ticket in tickets]
]
tickets = [int(elem) if elem.isdigit() else 9999999999 for elem in tickets]
test['Ticket'] = tickets

## <p style="background-color:Yellow; font-family:newtimeroman; font-size:160%; text-align:center; border-radius: 15px 50px;">5. Model Building and Validation</p>

In [ ]:
y = train.loc[:, ['Survived']]
X = train.drop(axis=1, columns=['Survived'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## <p style="background-color:magenta; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Stacking </p>

In [ ]:
# Defining base learners
myclf1 = KNeighborsClassifier(n_neighbors=1)
myclf2 = RandomForestClassifier(random_state=1)
myclf3 = GaussianNB()

In [ ]:
# Defining meta model
mylr = LogisticRegression()

In [ ]:
# creating stacking classifier with above models
stackingclf = StackingClassifier(estimators=[myclf1, myclf2, myclf3], final_estimator=mylr)

In [ ]:
print('Doing 3-fold cross validation here:\n')

for iterclf, iterlabel in zip([myclf1, myclf2, myclf3, stackingclf],
                              ['K-Nearest Neighbors Model',
                               'Random Forest Model',
                               'Naive Bayes Model',
                               'StackingClassifier Model']):
    scores = cross_val_score(iterclf, X_train,y_train, cv=3, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.3f) [%s]"
         % (scores.mean(), scores.std(), iterlabel))

In [ ]:
estimators = [
    ('rfc', RandomForestClassifier(max_depth=70, 
                                   n_estimators=58, 
                                   min_samples_split=2,
                                   n_jobs=-1,
                                   random_state=3)),
    ('lgbm', LGBMClassifier(max_depth=123,
                            n_estimators=95,
                            n_jobs=-1,
                            num_leaves=65,
                            random_state=95),
    ('xgb', XGBClassifier(max_depth=150, 
                          n_estimators=95, 
                          random_state=45, 
                          n_jobs=-1)))
]

In [ ]:
stack_clfs = StackingClassifier(estimators = estimators, 
                                final_estimator = LGBMClassifier())

In [ ]:
stack_clfs.fit(X_train, y_train)

In [ ]:
Y_pred = stack_clfs.predict(X_test)
accuracy_score(Y_pred, y_test)

In [ ]:
y_pred_stack_clfs = stack_clfs.predict(test)

## <p style="background-color:magenta; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Feature Importance </p>

In [ ]:
lgbm_parameters = {
    'reg_alpha': 0.00388218567052311,
    'reg_lambda': 8.972335390951376e-05,
    'colsample_bytree': 0.18375780999902297,
    'subsample': 0.013352256062576087,
    'learning_rate': 0.002597839272059483,
    'max_depth': 44,
    'num_leaves': 15,
    'min_child_samples': 89,
    'cat_smooth': 56, 
    'cat_l2': 22.375773634793603,
    'max_bin': 33, 
    'min_data_per_group': 89
}

In [ ]:
lgbm_parameters['metric'] = 'binary_logloss'
lgbm_parameters['objective'] = 'binary'
lgbm_parameters['n_estimators'] = 15000

In [ ]:
lgbm_model = LGBMClassifier(**lgbm_parameters)

In [ ]:
lgbm_model.fit(X_train, y_train)

In [ ]:
plt.rcParams["figure.figsize"] = (6, 5)
lightgbm.plot_importance(lgbm_model, max_num_features = 16, height=.9)

## <p style="background-color:magenta; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Submission </p>

In [ ]:
submission = pd.DataFrame({"PassengerId": testoriginal["PassengerId"], "Survived": y_pred_stack_clfs})
submission.to_csv('submission.csv', index=False)